In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

In [2]:
%matplotlib inline

In [3]:
import os
import numpy as np
import pandas as pd
from random import sample, seed
seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt

#-------------------------- set gpu using tf ---------------------------
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.Session(config=config)
#-------------------  start importing keras module ---------------------
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Flatten, Dense
from keras.models import Model
from keras import regularizers
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array
from keras.datasets import cifar10
from keras.callbacks import TensorBoard
# from sklearn.cross_validation import StratifiedShuffleSplit
from keras import models
from keras import optimizers
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from scipy.misc import imresize
import tensorflow as tf
from skimage.io import imread
from sklearn.model_selection import train_test_split
import keras.callbacks as callbacks

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
df = pd.read_csv('../fer2013/fer2013.csv', dtype={'emotion':np.int32, 'pixels':str, 'Usage':str})

In [5]:
df['pixels'] = df['pixels'].apply(lambda x: np.fromstring(x,sep=' '))

In [6]:
train = df.loc[df['Usage'] == 'Training']
validation = df.loc[df['Usage'] == 'PublicTest']
test = df.loc[df['Usage'] == 'PrivateTest']

y_train = pd.get_dummies(train['emotion'])
y_train.columns = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

y_val = pd.get_dummies(validation['emotion'])
y_val.columns = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

y_test = pd.get_dummies(test['emotion'])
y_test.columns = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

In [7]:
x_train = np.vstack(train['pixels'].values)
x_validation = np.vstack(validation['pixels'].values)
x_test = np.vstack(test['pixels'].values)

In [8]:
X_train = np.stack((np.reshape(x_train,(-1, 48, 48, 1)),)*3, axis=-2).squeeze()
X_val = np.stack((np.reshape(x_validation,(-1, 48, 48, 1)),)*3, axis=-2).squeeze()
X_test =  np.stack((np.reshape(x_test,(-1, 48, 48, 1)),)*3, axis=-2).squeeze()

In [9]:
def get_color(color_files, y, folder):
    numbers = []
    for item in color_files:
        for subitem in item.split('_'):
            if(subitem.isdigit()):
                numbers.append(subitem)

    X_color = np.empty((len(numbers), 48, 48, 3)).astype('uint8')
    y_color = np.empty((len(numbers), 7))

    for i in range(len(numbers)):

        path = folder + color_files[i]
        val = int(numbers[i])

        X_color[i] = np.array(imread(path)).astype('uint8')
        y_color[i] = y.iloc[val]
        
    return X_color, y_color, numbers

    #np.save('train_data', colorVal)

In [10]:
# train_color_files = [k for k in os.listdir('train_color/') if '_color' in k]

# X_train, y_train, indices_train = get_color(train_color_files, y_train, './train_color/')

In [11]:
# X_train, X_testval, y_train, y_testval = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [12]:
# X_val, X_test, y_val, y_test = train_test_split(X_testval, y_testval, test_size=0.5, random_state=0)

In [13]:
with tf.device('/cpu:0'):
    tf_x_train = tf.placeholder(tf.float32, shape=(len(X_train), 48, 48, 3))
    tf_x_validation = tf.placeholder(tf.float32, shape=(len(X_val), 48, 48, 3))
    tf_x_test = tf.placeholder(tf.float32, shape=(len(X_test), 48, 48, 3))

    tf_x_train_resized = tf.image.resize_images(tf_x_train,  size=[75,75])
    tf_x_validation_resized = tf.image.resize_images(tf_x_validation,  size=[75,75])
    tf_x_test_resized = tf.image.resize_images(tf_x_test,  size=[75,75])

In [14]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    X_train,X_val,X_test = sess.run([tf_x_train_resized,tf_x_validation_resized,tf_x_test_resized], feed_dict={tf_x_train: X_train,
                                                   tf_x_validation: X_val,
                                                   tf_x_test: X_test
                                                  })

In [15]:
np.shape(X_train)

(28709, 75, 75, 3)

In [16]:
datagen = ImageDataGenerator(rescale=1./255)

In [17]:
#datagen = ImageDataGenerator(
#    samplewise_center=True,
#    samplewise_std_normalization=True,
#    horizontal_flip=True,
#    vertical_flip=False)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train)

In [18]:
model_imagenet = applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(75, 75, 3))

In [19]:
model_imagenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 18, 128)       0         
__________

In [20]:
model_imagenet.layers[-1].name

'block5_pool'

In [21]:
#Add new classification layers
x = model_imagenet.layers[-1].output
x = Flatten()(x)
x = Dense(7)(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model = Model(model_imagenet.inputs, x, name='model_new')

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 18, 128)       0         
__________

In [23]:
#for layer in model.layers:
#    layer.trainable = True

for layer in model.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 True
block1_conv2 True
block1_pool True
block2_conv1 True
block2_conv2 True
block2_pool True
block3_conv1 True
block3_conv2 True
block3_conv3 True
block3_pool True
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True
flatten_1 True
dense_1 True
new_loss True


In [24]:
print(model_imagenet.output.shape)
print(model.output.shape)

(?, 2, 2, 512)
(?, 7)


Now, we compile our model and train it:

In [25]:
batch_size_val = 2**7

In [26]:
batch_size_val

128

In [27]:
tbCallBack = callbacks.TensorBoard(log_dir = "./tensorboard/")
early_stopping_callback = callbacks.EarlyStopping(monitor='val_acc', patience=5, mode='max')
checkpoint_callback = callbacks.ModelCheckpoint('vgg16_fine_tuning.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [28]:
# Compile model and train it.

model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr = 0.0001), metrics=["accuracy"])

history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size_val),
                              validation_data=datagen.flow(X_val, y_val, batch_size=batch_size_val), 
                              steps_per_epoch=len(X_train) / batch_size_val, epochs=100, 
                              validation_steps=len(X_val)/batch_size_val,
                              callbacks=[early_stopping_callback, checkpoint_callback, tbCallBack])

Epoch 1/100
225/224 [==============================] - 120s 534ms/step - loss: 1.3618 - acc: 0.4754 - val_loss: 1.1671 - val_acc: 0.5439

Epoch 00001: val_acc improved from -inf to 0.54388, saving model to vgg16_fine_tuning.h5
Epoch 2/100
225/224 [==============================] - 112s 498ms/step - loss: 1.0597 - acc: 0.5971 - val_loss: 1.0664 - val_acc: 0.5943

Epoch 00002: val_acc improved from 0.54388 to 0.59432, saving model to vgg16_fine_tuning.h5
Epoch 3/100
225/224 [==============================] - 112s 498ms/step - loss: 0.9303 - acc: 0.6502 - val_loss: 0.9874 - val_acc: 0.6347

Epoch 00003: val_acc improved from 0.59432 to 0.63472, saving model to vgg16_fine_tuning.h5
Epoch 4/100
225/224 [==============================] - 111s 495ms/step - loss: 0.8133 - acc: 0.6982 - val_loss: 0.9720 - val_acc: 0.6422

Epoch 00004: val_acc improved from 0.63472 to 0.64224, saving model to vgg16_fine_tuning.h5
Epoch 5/100
225/224 [==============================] - 112s 496ms/step - loss: 0.69

Finally, let's evaluate on our test set:

In [29]:
# Evaluate on validation:
# ...
print(model.metrics_names)
print(model.evaluate_generator(datagen.flow(X_val, y_val, batch_size=batch_size_val), steps=len(X_val)/batch_size_val))

['loss', 'acc']
[2.0594294728193816, 0.6678740596598521]


In [30]:
# Evaluate on test:
# ...
print(model.metrics_names)
print(model.evaluate_generator(datagen.flow(X_test, y_test, batch_size=batch_size_val), steps=len(X_test)/batch_size_val))

['loss', 'acc']
[1.9681121640698296, 0.6792978545638904]
